Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [10]:
def model(X, w_h, b_h, w_h2, b_h2, w_o, b_o, p_keep_input, p_keep_hidden): 

    X = tf.nn.dropout(X, p_keep_input)
    h = tf.nn.relu(tf.matmul(X, w_h)+b_h)

    h = tf.nn.dropout(h, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h, w_h2)+b_h2)

    h2 = tf.nn.dropout(h2, p_keep_hidden)

    return tf.matmul(h2, w_o)+b_o

def model_det(X, w_h, b_h, w_h2, b_h2, w_o, b_o): 

    h = tf.nn.relu(tf.matmul(X, w_h)+b_h)
    h2 = tf.nn.relu(tf.matmul(h, w_h2)+b_h2)

    return tf.matmul(h2, w_o)+b_o

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def myaccuracy(u, v):
  return 100.0*np.sum(u == np.argmax(v, 1))/u.shape[0]

batch_size = 512

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  X = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  Y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  X_val = tf.constant(valid_dataset)
  X_tst = tf.constant(test_dataset)

  # Variables.
  w_h = init_weights([784, 625])
  b_h = init_weights([625])
  w_h2 = init_weights([625, 625])
  b_h2 = init_weights([625])
  w_o = init_weights([625, 10])
  b_o = init_weights([10])

  p_keep_input = tf.constant(0.9)
  p_keep_hidden = tf.constant(0.75)

  py_x = model(X, w_h, b_h, w_h2, b_h2, w_o, b_o, p_keep_input, p_keep_hidden)
  py_xtest = model_det(X_tst, w_h, b_h, w_h2, b_h2, w_o, b_o)
  py_xval = model_det(X_val, w_h, b_h, w_h2, b_h2, w_o, b_o)
  
  
  # Add the regularization term to the loss.
  regularizers = (tf.nn.l2_loss(w_h) + tf.nn.l2_loss(b_h) +
                  tf.nn.l2_loss(w_h2) + tf.nn.l2_loss(b_h2) +
                  tf.nn.l2_loss(w_o) + tf.nn.l2_loss(b_o))
  
  # Training computation
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y)) + 5e-4*regularizers
  optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.argmax(py_x, 1)
  valid_prediction = tf.argmax(py_xval, 1)
  test_prediction = tf.argmax(py_xtest, 1)

In [11]:
num_steps = 15001
from random import sample

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {X : batch_data, Y : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      u, v = predictions, batch_labels
      #print("Minibatch accuracy: %.1f%%" % myaccuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % myaccuracy(valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % myaccuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.325187
Validation accuracy: 9.9%
Test accuracy: 9.7%
Minibatch loss at step 500: 0.607441
Validation accuracy: 85.3%
Test accuracy: 91.4%
Minibatch loss at step 1000: 0.415126
Validation accuracy: 87.7%
Test accuracy: 93.4%
Minibatch loss at step 1500: 0.492149
Validation accuracy: 88.8%
Test accuracy: 94.2%
Minibatch loss at step 2000: 0.440133
Validation accuracy: 89.0%
Test accuracy: 94.3%
Minibatch loss at step 2500: 0.406215
Validation accuracy: 89.4%
Test accuracy: 94.7%
Minibatch loss at step 3000: 0.448152
Validation accuracy: 89.1%
Test accuracy: 94.3%
Minibatch loss at step 3500: 0.451794
Validation accuracy: 89.3%
Test accuracy: 94.7%
Minibatch loss at step 4000: 0.424621
Validation accuracy: 89.5%
Test accuracy: 95.0%
Minibatch loss at step 4500: 0.475135
Validation accuracy: 88.5%
Test accuracy: 93.8%
Minibatch loss at step 5000: 0.461938
Validation accuracy: 89.3%
Test accuracy: 94.8%
Minibatch loss at step 5500: 0.543412
Validation

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens? 
### Overfitting.

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
